In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# ========== python ==========
import os
from pathlib import Path
from logging import Logger
from typing import List, Dict, Tuple, Optional, Union, Callable, Final, Literal, get_args
from operator import itemgetter, attrgetter
import itertools

from utils.setup import setup_logger, get_device
from const.const_values import PROJECT_DIR

os.chdir(PROJECT_DIR)
logger: Logger = setup_logger(__name__, f'{PROJECT_DIR}/log/jupyter_run.log')
device = get_device(device_name='cpu', logger=logger)
print(PROJECT_DIR)

/var/www


In [2]:
# jupyter
import seaborn as sns
import matplotlib.pyplot as plt
# Machine learning
import numpy as np
import pandas as pd
import h5py
import optuna
# torch
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard.writer import SummaryWriter
# torch ignite
from ignite.engine import Engine
from ignite.handlers import Checkpoint
# My items
from models.datasets.data_helper import MyDataHelper, MyDataLoaderHelper, DefaultTokens
from models.datasets.datasets_for_story import StoryTriple
# My utils
from utils.setup import load_param
from utils.torch import load_model, torch_fix_seed
# main function
from run_for_KGC import main_function


In [3]:
from const.const_values import CPU, MODEL
from models.KGModel.kg_model import HEAD, RELATION, TAIL
from utils.torch_ignite import TRAINER, EVALUATOR
from const.const_values import DATASETS, DATA_HELPER, DATA_LOADERS, TRAIN_RETURNS

In [4]:
SEED: Final[int] = 42
args_path = f'{PROJECT_DIR}/models/kgc03a/all100/param.pkl'
model_path = f'{PROJECT_DIR}/models/kgc03a/all100/model.pth'

In [5]:
args = load_param(args_path)

# args.pre_train = True
args.logger = logger
args.device = device
args.batch_size = 1
args.pre_train=False
args.init_embedding_using_bert = False
args.model_path = model_path
args.only_load_trainer_evaluator = True
args.old_data = 1

del args.optuna_file, args.device_name, args.pid, args.study_name, args.n_trials
logger.info(args)

2023-02-03 00:49:15 - INFO - 89188924.py - 14 - Namespace(notebook=False, console_level='info', logfile='models/kgc/all100/03/log.log', param_file='models/kgc/all100/03/param.pkl', train_anyway=False, tensorboard_dir='models/kgc/all100/03/tensorboard', checkpoint_dir='models/kgc/all100/03/checkpoint/', model_path='/var/www/models/kgc03a/all100/model.pth', resume_from_checkpoint=False, resume_from_last_point=False, only_load_trainer_evaluator=True, resume_checkpoint_path=None, pre_train=False, train_valid_test=False, only_train=True, use_for_challenge100=False, use_for_challenge090=False, use_for_challenge075=False, use_title=None, do_optuna=False, story_special_num=5, relation_special_num=5, entity_special_num=5, padding_token_e=0, cls_token_e=1, mask_token_e=2, sep_token_e=3, bos_token_e=4, padding_token_r=0, cls_token_r=1, mask_token_r=2, sep_token_r=3, bos_token_r=4, padding_token_s=0, cls_token_s=1, mask_token_s=2, sep_token_s=3, bos_token_s=4, model_version='03', embedding_dim=128

In [6]:
torch_fix_seed(seed=SEED)
return_dict = main_function(args, logger=logger)

2023-02-03 00:49:20 - INFO - run_for_KGC.py - 889 - ----- make datahelper start. -----
2023-02-03 00:49:20 - INFO - run_for_KGC.py - 587 - ----- use old data (version 1) -----
2023-02-03 00:49:20 - INFO - data_helper.py - 333 - ====== Show MyRawData ======
2023-02-03 00:49:20 - INFO - data_helper.py - 334 - entity num: 7844
2023-02-03 00:49:20 - INFO - data_helper.py - 335 - relation num: 63
2023-02-03 00:49:20 - INFO - data_helper.py - 336 - ====== Show MyRawData ======
2023-02-03 00:49:20 - INFO - data_helper.py - 607 - ========== Show DataHelper ==========
2023-02-03 00:49:20 - INFO - data_helper.py - 333 - ====== Show MyRawData ======
2023-02-03 00:49:20 - INFO - data_helper.py - 334 - entity num: 7844
2023-02-03 00:49:20 - INFO - data_helper.py - 335 - relation num: 63
2023-02-03 00:49:20 - INFO - data_helper.py - 336 - ====== Show MyRawData ======
2023-02-03 00:49:20 - INFO - data_helper.py - 609 - entity_special_dicts: {0: '<pad_e>', 1: '<cls_e>', 2: '<mask_e>', 3: '<sep_e>', 4:

In [7]:
model = return_dict[MODEL]

dataset_train: StoryTriple = return_dict[DATASETS][0]
triple: torch.Tensor = dataset_train.triple
data_helper: MyDataHelper = return_dict[DATA_HELPER]
evaluator: Checkpoint = return_dict[TRAIN_RETURNS][EVALUATOR]

load_model(model, args.model_path, device)
model.eval()

entities, relations = data_helper.processed_entities, data_helper.processed_relations
d_e, d_r = {e: i for i, e in enumerate(entities)}, {r: i for i, r in enumerate(relations)}

triple_df = pd.DataFrame([(entities[_t[0]], relations[_t[1]], entities[_t[2]]) for _t in triple], columns=[HEAD, RELATION, TAIL])
story_entities = triple_df[HEAD].tolist()

[('<pad_e>', 0), ('<cls_e>', 1), ('<mask_e>', 2), ('<sep_e>', 3), ('<bos_e>', 4), ('AbbeyGrange:Comparable_Officer', 7), ('AbbeyGrange:Burglary_team_of_Lewisham', 8), ('AbbeyGrange:Maid', 9), ('AbbeyGrange:everyone', 10), ('AbbeyGrange:Only_one_woman', 11), ('AbbeyGrange:Mary', 12), ('AbbeyGrange:The_local_police', 13), ('AbbeyGrange:maid', 14), ('AbbeyGrange:eight_maids', 15), ('AbbeyGrange:Old_man', 16), ('AbbeyGrange:bracket', 17), ('AbbeyGrange:appearance', 18), ('AbbeyGrange:Sir_Eustace_Brackenstall', 19), ('AbbeyGrange:use', 20), ('AbbeyGrange:False_charge_person', 21), ('AbbeyGrange:Sir_Eustace_brackenstall', 22), ('AbbeyGrange:Jack_Croker', 23), ('AllTitle:Watson', 24), ('AbbeyGrange:Theresa', 25), ('AllTitle:Holmes', 26), ('AbbeyGrange:The_Bays_Rock', 27), ('AbbeyGrange:criminal', 28), ('AbbeyGrange:Criminal', 29), ('AbbeyGrange:three_people', 30), ('AbbeyGrange:Train_crew', 31), ('AbbeyGrange:Lady_Brackenstall', 32), ('AbbeyGrange:siblings_of_Lady_Brackenstall', 33), ('AbbeyG

In [8]:
triple_df

,head,relation,tail
0,<bos_e>,<bos_r>,<bos_e>
1,AbbeyGrange:001,kgc:subject,AllTitle:Holmes
2,AbbeyGrange:001,kgc:hasPredicate,word.predicate:wakeUp
3,AbbeyGrange:001,kgc:whom,AllTitle:Watson
4,AbbeyGrange:001,kgc:infoSource,AllTitle:Watson
...,...,...,...
17886,AbbeyGrange:084,kgc:subject,AbbeyGrange:Theresa
17887,AbbeyGrange:084,kgc:hasPredicate,word.predicate:rest
17888,AbbeyGrange:084,kgc:where,AbbeyGrange:room_of_Lady_Brackenstall
17889,<bos_e>,<bos_r>,<bos_e>


In [9]:
def extract(model, target, inputs):
    feature = None

    def forward_hook(module, inputs, _):
        global features
        x, _, _ = inputs
        outputs = module.forward(x, x, x, need_weights=True)[1]
        features = outputs.detach().clone()

    handle = target.register_forward_hook(forward_hook)

    model.eval()
    model(inputs, torch.LongTensor([[]]), torch.LongTensor([[]]), torch.LongTensor([[]]))

    handle.remove()

    return features

In [10]:
def get_attention(input_):
    assert len(input_) == 1
    features = extract(model, model.transformer.layers[-1].self_attn, input_)[0]
    df_attention = pd.DataFrame([[entities[h], relations[r], entities[t]]+[features[j, i].item() for j in range(len(features))] for i, (h, r, t) in enumerate(input_[0])])
    df_attention.columns=[HEAD, RELATION, TAIL] + [f'atten_from{i}' for i in range(len(df_attention.columns)-3)]
    return df_attention

def show_attension_heatmap(df_attention):
    sns.heatmap(df_attention.iloc[:,3:])
    plt.show()

In [11]:
bos_triple = [d_e[DefaultTokens.BOS_E], d_r[DefaultTokens.BOS_R],d_e[DefaultTokens.BOS_E]]
mask_e_id = d_e[DefaultTokens.MASK_E]
Holmes_id =d_e['AllTitle:Holmes']

def make_ranking(from_story_name, to_story_name, predicate_, whom_, subject_, why_, what_, where_):
    if not (from_story_name is None and to_story_name is None):
        _start_index = story_entities.index(from_story_name)-1 
        _end_index = len(story_entities) - story_entities[::-1].index(to_story_name)
    else: 
        _start_index = 0
        _end_index = 0
    question_ = torch.tensor(
        [
            bos_triple, 
            [mask_e_id, d_r['kgc:infoSource'],     Holmes_id      ],
            [mask_e_id, d_r['kgc:hasPredicate'],   d_e[predicate_]],
            [mask_e_id, d_r['kgc:whom'],           d_e[whom_     ]],
            [mask_e_id, d_r['kgc:subject'],        d_e[subject_  ]],
            [mask_e_id, d_r['kgc:why'],            d_e[why_      ]],
            [mask_e_id, d_r['kgc:what'],           d_e[what_     ]],
            [mask_e_id, d_r['kgc:where'],          d_e[where_    ]],
        ]
    )
    mask_ = torch.zeros_like(question_, dtype=torch.bool) # not mask all position
    mask_[1:, 0] = True                                   # where head position without bos token
    mask_[1:, 2] = True                                   # where tail position without bos token

    last_triples = triple[_start_index: _end_index]

    questions = torch.cat([last_triples, question_], dim=0).unsqueeze(0)
    masks = torch.cat([torch.zeros_like(last_triples), mask_], dim=0).to(torch.bool).transpose(1,0).unsqueeze(0)

    data_list = []
    with torch.no_grad():
        _, (story_pred, relation_pred, entity_pred) = model(questions, masks[:,0], masks[:,1], masks[:,2])
        sorted_ = torch.argsort(entity_pred, dim=1, descending=True)
        for i in range(sorted_.shape[1]):
            ans_= sorted_[:, i]
            info_source_, predicate_pred, whom_pred, subject_pred, why_pred, what_pred, where_pred = ans_
            data_list.append([entities[predicate_pred], entities[whom_pred], entities[subject_pred], entities[why_pred], entities[what_pred], entities[where_pred]])
    df_ranking = pd.DataFrame(data_list, columns=['predicate', 'whom', 'subject', 'why', 'what', 'where'])
    df_attension = get_attention(questions)
    
    return df_ranking, df_attension

In [12]:
MASK_E = DefaultTokens.MASK_E
KILL = 'word.predicate:kill'

def main_func01(_title, _victim_name, criminal, predicate, _last_index, _story_len):
    from_ = f'{_title}:{_last_index-_story_len+1}'
    to_ = f'{_title}:{_last_index}'
    predicate = predicate
    victim = f'{_title}:{_victim_name}'
    criminal = f'{_title}:{criminal}'
    df_ranking, df_attension = make_ranking(
        from_, to_, predicate, victim, MASK_E, MASK_E, MASK_E, MASK_E)

    pred_rank = df_ranking.index[df_ranking['subject']==criminal].tolist()[0]
    logger.info(f"The pred ranking about {criminal} is {pred_rank}")
    display(df_ranking.iloc[:max(20, pred_rank)])
    len_ = len(df_attension)
    for i in range(len_-10, len_):
        display(i, df_attension.iloc[i,:3].tolist())
        display(df_attension.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
        print("----------")
    return df_ranking, df_attension

def check_killer(_title, _victim_name, _killer_name, _last_index, _story_len):
    return main_func01(_title, _victim_name, _killer_name, KILL, _last_index, _story_len)

In [13]:
TAKE = 'word.predicate:take'
BRING = 'word.predicate:bring'
DIE = 'word.predicate:die'
HIDE = 'word.predicate:hide'

### まだらの紐
Who killed Julia? (criminal & explanation)
被害者: Julia
犯人: Roylott
犯行に用いたもの: snake
犯行動機: 母の相続財産を独占したい

In [14]:
def do_madara_pred():
    title = 'SpeckledBand'
    victim_name = 'Julia'
    killer_name = 'Roylott'
    last_index = 401
    story_len = 80

    df_ranking_SpeckledBand, df_attention_SpeckledBand = check_killer(title, victim_name, killer_name, last_index, story_len)
do_madara_pred()

2023-02-02 17:26:43 - INFO - 2402927636.py - 14 - The pred ranking about SpeckledBand:Roylott is 3


,predicate,whom,subject,why,what,where
0,word.predicate:distort,AbbeyGrange:Sound_of_bell,SpeckledBand:mother_of_Helen,SpeckledBand:left_building,SpeckledBand:Roylott,SpeckledBand:dog_whip_of_Roylott
1,word.predicate:confirm,AbbeyGrange:Burglary_team_of_Lewisham,SpeckledBand:sleepwear,SpeckledBand:price_of_asset_of_mother_of_Helen,SpeckledBand:sleepwear,SpeckledBand:left_building
2,AbbeyGrange:Sound_of_bell,AbbeyGrange:Two_people,SilverBlaze:Silas_Brown,AbbeyGrange:Two_people,SpeckledBand:Indian_animal,SpeckledBand:hole
3,DancingMen:Horse_boy,SpeckledBand:sleepwear,SpeckledBand:Roylott,SpeckledBand:Indian_animal,SpeckledBand:culprit,AbbeyGrange:Brutal_character_of_criminal
4,word.predicate:believe,SilverBlaze:Lord_Backwater,AbbeyGrange:Burglary_team_of_Lewisham,ResidentPatient:Percy_Trevelyan,SpeckledBand:left_building,SpeckledBand:Indian_animal
5,word.predicate:enjoy,SpeckledBand:Roylott,AbbeyGrange:All_of_maids,SpeckledBand:Roylott,AbbeyGrange:Two_people,AbbeyGrange:All_of_maids
6,AbbeyGrange:Burglary_team_of_Lewisham,SpeckledBand:left_building,DevilsFoot:Former_wife,AbbeyGrange:Burglary_team_of_Lewisham,SpeckledBand:price_of_asset_of_mother_of_Helen,AbbeyGrange:Two_people
7,word.predicate:perform,SpeckledBand:mother_of_Helen,AbbeyGrange:Two_people,AbbeyGrange:All_of_maids,DateTime:1883-04-01T15:00:00,CrookedMan:Henry
8,word.predicate:leaveBehind,SpeckledBand:scar,DancingMen:Maid,AbbeyGrange:maid,SpeckledBand:by_falling_price,AbbeyGrange:Burglary_team_of_Lewisham
9,CrookedMan:someone,SpeckledBand:hole,SpeckledBand:Roma,SpeckledBand:elbow_chair,AbbeyGrange:appearance,AbbeyGrange:appearance


452

['SpeckledBand:401', 'kgc:subject', 'SpeckledBand:Roma']

,head,relation,tail,atten_from452
444,SpeckledBand:399,kgc:time,DateTime:1883-04-01T23:00:00,0.023507
452,SpeckledBand:401,kgc:subject,SpeckledBand:Roma,0.021681
328,SpeckledBand:377,kgc:subject,SpeckledBand:Roylott,0.016120
401,SpeckledBand:391,kgc:time,DateTime:1881-04-02T00:30:00,0.014584
450,SpeckledBand:400,kgc:time,DateTime:1883-04-01T23:00:00,0.013840
412,SpeckledBand:394,kgc:subject,SpeckledBand:Helen,0.012100
438,SpeckledBand:398,kgc:time,DateTime:1883-04-01T23:00:00,0.012021
448,SpeckledBand:400,kgc:what,SpeckledBand:Roylott,0.009957
335,SpeckledBand:378,kgc:subject,SpeckledBand:coroner,0.009874
294,SpeckledBand:371,kgc:time,DateTime:1883-04-01T15:00:00,0.009822


...

----------


461

['<mask_e>', 'kgc:where', '<mask_e>']

,head,relation,tail,atten_from461
401,SpeckledBand:391,kgc:time,DateTime:1881-04-02T00:30:00,0.011362
300,SpeckledBand:372,kgc:time,DateTime:1883-04-01T15:00:00,0.009855
299,SpeckledBand:372,kgc:when,SpeckledBand:1883-04-01T15,0.009833
294,SpeckledBand:371,kgc:time,DateTime:1883-04-01T15:00:00,0.008576
120,SpeckledBand:341,kgc:time,DateTime:1883-04-02T04:00:00,0.008201
127,SpeckledBand:343,kgc:time,DateTime:1883-04-02T04:00:00,0.007720
304,SpeckledBand:373,kgc:when,SpeckledBand:1883-04-01T15,0.007570
444,SpeckledBand:399,kgc:time,DateTime:1883-04-01T23:00:00,0.007448
438,SpeckledBand:398,kgc:time,DateTime:1883-04-01T23:00:00,0.007446
344,SpeckledBand:379,kgc:time,DateTime:1873-04-01T15:00:00,0.007426


----------


### 悪魔の足跡１
Who killed the victims? (criminal & explanation)
被害者: Brenda
犯人: Mortimer
犯行動機: 財産

In [15]:
def do_devil1_pred():
    title = 'DevilsFoot'
    victim_name = 'Brenda'
    killer_name = 'Mortimer'
    last_index = 489
    story_len = 80
    df_ranking_DevilsFoot1, df_attension_DevilsFoot1 = check_killer(title, victim_name, killer_name, last_index, story_len)
do_devil1_pred()

2023-01-22 03:29:50 - INFO - 2822292237.py - 14 - The pred ranking about DevilsFoot:Mortimer is 6


,predicate,whom,subject,why,what,where
0,word.predicate:fire,DevilsFoot:Owen,DevilsFoot:Expression_of_fear,DevilsFoot:Expression_of_fear,DevilsFoot:Expression_of_fear,DevilsFoot:anything
1,word.predicate:close,DevilsFoot:Expression_of_fear,DevilsFoot:Owen,DevilsFoot:Owen,DevilsFoot:Owen,DevilsFoot:Expression_of_fear
2,word.predicate:stay,DevilsFoot:Doctor_Richard,DevilsFoot:Doctor_Richard,<bos_e>,DevilsFoot:Doctor_Richard,DevilsFoot:Sense_of_guilt
3,word.predicate:confirm,DevilsFoot:Former_wife,AbbeyGrange:Theresa,DateTime:1890-04-01T00:00:00,DevilsFoot:Path,DevilsFoot:Good_mental_condition
4,word.predicate:notNotice,<bos_e>,DevilsFoot:Former_wife,DevilsFoot:garden,DevilsFoot:Former_wife,DevilsFoot:Path
5,word.predicate:stuck,DevilsFoot:George,DevilsFoot:George,DevilsFoot:Doctor_Richard,ACaseOfIdentity:Two_letters,DevilsFoot:Owen
6,word.predicate:notInterfere,AbbeyGrange:Theresa,DevilsFoot:Mortimer,DevilsFoot:This_afternoon,<bos_e>,DevilsFoot:sample_of_magic_foot
7,word.predicate:catch,DevilsFoot:Mortimer,AbbeyGrange:acrobat,DevilsFoot:Former_wife,DevilsFoot:George,DevilsFoot:Failure
8,word.predicate:takeOut,ACaseOfIdentity:Man,SpeckledBand:housekeeper,DevilsFoot:anything,ACaseOfIdentity:Man,DevilsFoot:Jury_member
9,word.predicate:suicide,DevilsFoot:Brenda,DevilsFoot:Garden_path,DevilsFoot:Path,DevilsFoot:Mortimer,DevilsFoot:Doctor_Richard


358

['DevilsFoot:489', 'kgc:hasPredicate', 'word.predicate:find']

,head,relation,tail,atten_from358
351,DevilsFoot:487,kgc:hasProperty,word.predicate:gravel,0.100623
314,DevilsFoot:477,kgc:hasPredicate,word.predicate:paralyze,0.040914
125,DevilsFoot:436,kgc:hasProperty,word.predicate:good_friends,0.039822
352,DevilsFoot:487,kgc:on,DevilsFoot:frame_of_Window,0.037219
305,DevilsFoot:475,kgc:hasProperty,word.predicate:Executioner,0.032367
86,DevilsFoot:426,kgc:whom,DevilsFoot:Sterndale,0.020472
350,DevilsFoot:487,kgc:subject,DevilsFoot:Starting_point_of_investigation,0.018768
83,DevilsFoot:426,kgc:subject,DevilsFoot:Roundhay,0.016308
213,DevilsFoot:455,kgc:whom,AllTitle:Holmes,0.015406
2,DevilsFoot:410,kgc:hasPredicate,word.predicate:return,0.012342


...

----------


367

['<mask_e>', 'kgc:where', '<mask_e>']

,head,relation,tail,atten_from367
290,DevilsFoot:473,kgc:what,DevilsFoot:477,0.033087
287,DevilsFoot:473,kgc:what,DevilsFoot:474,0.028712
289,DevilsFoot:473,kgc:what,DevilsFoot:476,0.025475
283,DevilsFoot:472,kgc:whom,DevilsFoot:Mortimer,0.024490
288,DevilsFoot:473,kgc:what,DevilsFoot:475,0.023681
294,DevilsFoot:473,kgc:what,DevilsFoot:481,0.023669
292,DevilsFoot:473,kgc:what,DevilsFoot:479,0.020932
296,DevilsFoot:473,kgc:what,DevilsFoot:484,0.018948
295,DevilsFoot:473,kgc:what,DevilsFoot:483,0.018769
293,DevilsFoot:473,kgc:what,DevilsFoot:480,0.014288


----------


### 悪魔の足跡2
Who killed the victims? (criminal & explanation)
被害者: Mortimer
犯人: 
犯行動機: 恋人の敵

In [16]:
def do_devil2_pred():
    title = 'DevilsFoot'
    victim_name = 'Mortimer'
    killer_name = 'Sterndale'
    last_index = 489
    story_len = 80
    df_ranking_DevilsFoot2, df_attension_DevilsFoot2 = check_killer(title, victim_name, killer_name, last_index, story_len)
do_devil2_pred()

2023-01-22 03:29:51 - INFO - 2822292237.py - 14 - The pred ranking about DevilsFoot:Sterndale is 1319


,predicate,whom,subject,why,what,where
0,word.predicate:fire,DevilsFoot:Owen,DevilsFoot:Expression_of_fear,DevilsFoot:Expression_of_fear,DevilsFoot:Expression_of_fear,DevilsFoot:anything
1,word.predicate:close,DevilsFoot:Expression_of_fear,DevilsFoot:Owen,DevilsFoot:Owen,DevilsFoot:Owen,DevilsFoot:Expression_of_fear
2,word.predicate:confirm,DevilsFoot:Doctor_Richard,DevilsFoot:Doctor_Richard,<bos_e>,DevilsFoot:Doctor_Richard,DevilsFoot:Sense_of_guilt
3,word.predicate:stay,ACaseOfIdentity:Man,AbbeyGrange:Theresa,DevilsFoot:Doctor_Richard,<bos_e>,DevilsFoot:Good_mental_condition
4,word.predicate:notInterfere,<bos_e>,DevilsFoot:Former_wife,DevilsFoot:Former_wife,ACaseOfIdentity:Two_letters,DevilsFoot:Owen
...,...,...,...,...,...,...
1314,SilverBlaze:garden_party,DevilsFoot:255,DancingMen:Bookshelf,SpeckledBand:window,ResidentPatient:055,DevilsFoot:Face_to_face
1315,DevilsFoot:death_of_Mortimer,CrookedMan:police,word.predicate:end,ResidentPatient:Moderator_of_Tobin,word.predicate:stick,word.predicate:Ring
1316,DevilsFoot:guilt_of_Mortimer,SpeckledBand:130,DateTime:1891-08-29T00:00:00,DateTime:1887-07-06T20:55:00,SilverBlaze:359,SilverBlaze:brown_color
1317,ACaseOfIdentity:character,CrookedMan:Nancy,SilverBlaze:058a,ACaseOfIdentity:196,AbbeyGrange:Maid,word.predicate:wearOut


358

['DevilsFoot:489', 'kgc:hasPredicate', 'word.predicate:find']

,head,relation,tail,atten_from358
351,DevilsFoot:487,kgc:hasProperty,word.predicate:gravel,0.100261
314,DevilsFoot:477,kgc:hasPredicate,word.predicate:paralyze,0.040851
125,DevilsFoot:436,kgc:hasProperty,word.predicate:good_friends,0.040351
352,DevilsFoot:487,kgc:on,DevilsFoot:frame_of_Window,0.037192
305,DevilsFoot:475,kgc:hasProperty,word.predicate:Executioner,0.032131
86,DevilsFoot:426,kgc:whom,DevilsFoot:Sterndale,0.020876
350,DevilsFoot:487,kgc:subject,DevilsFoot:Starting_point_of_investigation,0.018010
83,DevilsFoot:426,kgc:subject,DevilsFoot:Roundhay,0.015956
213,DevilsFoot:455,kgc:whom,AllTitle:Holmes,0.014928
2,DevilsFoot:410,kgc:hasPredicate,word.predicate:return,0.012347


...

----------


367

['<mask_e>', 'kgc:where', '<mask_e>']

,head,relation,tail,atten_from367
290,DevilsFoot:473,kgc:what,DevilsFoot:477,0.034246
287,DevilsFoot:473,kgc:what,DevilsFoot:474,0.028982
289,DevilsFoot:473,kgc:what,DevilsFoot:476,0.025577
283,DevilsFoot:472,kgc:whom,DevilsFoot:Mortimer,0.024440
288,DevilsFoot:473,kgc:what,DevilsFoot:475,0.024161
294,DevilsFoot:473,kgc:what,DevilsFoot:481,0.023463
292,DevilsFoot:473,kgc:what,DevilsFoot:479,0.020083
295,DevilsFoot:473,kgc:what,DevilsFoot:483,0.019753
296,DevilsFoot:473,kgc:what,DevilsFoot:484,0.018909
273,DevilsFoot:471,kgc:subject,DevilsFoot:Mortimer,0.014534


----------


### 背中の曲がった男: 
Why did Barclay die?
被害者: Barclay
犯人: 
犯行動機: 

In [17]:
victim = 'CrookedMan:Barclay'
df_ranking_CrookedMan, df_attension_CrookedMan = make_ranking(
    f'CrookedMan:{373-80+1}', 'CrookedMan:373', DIE, MASK_E, victim, MASK_E, MASK_E, MASK_E)

display(df_ranking_CrookedMan.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_CrookedMan)
for i in range(len_-20, len_):
    display(i, df_attension_CrookedMan.iloc[i,:3].tolist())
    display(df_attension_CrookedMan.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

,predicate,whom,subject,why,what,where
0,word.predicate:close,CrookedMan:father_of_Nancy,CrookedMan:the_St._George_union,CrookedMan:Simpson,CrookedMan:the_St._George_union,CrookedMan:rarely
1,word.predicate:suicide,CrookedMan:Monday,CrookedMan:father_of_Nancy,CrookedMan:body,CrookedMan:body,AbbeyGrange:Three_times
2,word.predicate:notHear,CrookedMan:coachman,AbbeyGrange:Theresa,CrookedMan:father_of_Nancy,CrookedMan:Simpson,CrookedMan:Old_Testament
3,word.predicate:putOut,DancingMen:Martin,CrookedMan:coachman,CrookedMan:1887-07-06T21:15:00,CrookedMan:rarely,SpeckledBand:scar
4,word.predicate:report,CrookedMan:the_St._George_union,CrookedMan:coward,CrookedMan:rarely,CrookedMan:coachman,CrookedMan:father_of_Nancy
5,word.predicate:cost,CrookedMan:Simpson,CrookedMan:Simpson,DancingMen:the_first_position,CrookedMan:father_of_Nancy,CrookedMan:the_St._George_union
6,word.predicate:swell,AbbeyGrange:Theresa,CrookedMan:Monday,CrookedMan:mongoose,CrookedMan:dwellers_of_mountain,word.predicate:drawn
7,word.predicate:canGetAhead,CrookedMan:coward,CrookedMan:dwellers_of_mountain,CrookedMan:the_St._George_union,CrookedMan:many_times,CrookedMan:Simpson
8,word.predicate:notLeave,DancingMen:Maid,DancingMen:Maid,word.predicate:drawn,DancingMen:Maid,CrookedMan:mongoose
9,word.predicate:hop,AbbeyGrange:acrobat,CrookedMan:doctor,<http://kgc.knowledge-graph.jp/data/DancingMen...,CrookedMan:soldier,CrookedMan:coward


334

['CrookedMan:371', 'kgc:subject', 'CrookedMan:Barclay']

,head,relation,tail,atten_from334
321,CrookedMan:368,kgc:what,CrookedMan:369,0.058586
322,CrookedMan:368,kgc:what,CrookedMan:370,0.036129
280,CrookedMan:360,kgc:subject,CrookedMan:Barclay,0.026750
323,CrookedMan:368,kgc:what,CrookedMan:371,0.023696
139,CrookedMan:324,kgc:subject,CrookedMan:Nancy,0.019932
182,CrookedMan:332,kgc:subject,CrookedMan:Henry,0.015516
238,CrookedMan:348,kgc:what,CrookedMan:349,0.014941
232,CrookedMan:346,kgc:what,CrookedMan:347,0.014362
67,CrookedMan:309,kgc:what,CrookedMan:318,0.013829
122,CrookedMan:318a,kgc:what,CrookedMan:086,0.013666


----------


...

----------


353

['<mask_e>', 'kgc:where', '<mask_e>']

,head,relation,tail,atten_from353
124,CrookedMan:320,kgc:subject,CrookedMan:Henry,0.038120
238,CrookedMan:348,kgc:what,CrookedMan:349,0.019288
236,CrookedMan:348,kgc:subject,CrookedMan:Henry,0.013705
90,CrookedMan:312,kgc:subject,CrookedMan:Henry,0.013624
280,CrookedMan:360,kgc:subject,CrookedMan:Barclay,0.012941
117,CrookedMan:318,kgc:infoSource,AllTitle:Holmes,0.012723
121,CrookedMan:318a,kgc:subject,CrookedMan:Henry,0.011352
118,CrookedMan:318,kgc:when,CrookedMan:1887-07-06T21:15:00,0.011254
291,CrookedMan:363,kgc:subject,AllTitle:Holmes,0.010785
182,CrookedMan:332,kgc:subject,CrookedMan:Henry,0.010456


----------


### 僧坊荘園
Who killed Lord Blackenstall? (criminal & explanation)
被害者: Sir_Eustace_Brackenstall
犯人: 
犯行動機: 

In [18]:
title = 'AbbeyGrange'
victim_name = 'Sir_Eustace_Brackenstall'
killer_name = 'Jack_Croker'
last_index = 414
story_len = 80

df_ranking_AbbeyGrange, df_attension_AbbeyGrange = main_func01(title, victim_name, killer_name, last_index, story_len)

TypeError: main_func01() missing 1 required positional argument: '_story_len'

### 入院患者
Who killed Blessington? (criminal & explanation)
被害者: Blessington
犯人: 3人
犯行動機: 

In [ ]:
title = 'ResidentPatient'
victim_name = 'Blessington'
killer_name = ''
last_index = 324
story_len = 80

df_ranking_Blessington, df_attension_Blessington = main_func01(title, victim_name, killer_name, last_index, story_len)

### 白銀
Who took out the White Silver Blaze? (criminal & explanation) 
被害者: Silver_Blaze
犯人: 
犯行動機: 

In [ ]:
victim = 'SilverBlaze:Silver_Blaze'
df_ranking_SilverBlaze, df_attension_SilverBlaze = make_ranking(
    'SilverBlaze:330', 'SilverBlaze:396', BRING, MASK_E, MASK_E, MASK_E, victim, MASK_E)

display(df_ranking_SilverBlaze.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_SilverBlaze)
for i in range(len_-20, len_):
    display(i, df_attension_SilverBlaze.iloc[i,:3].tolist())
    display(df_attension_SilverBlaze.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")

### 花嫁失踪事件（同一事件）
hozmaの失踪の名瀬を探る
被害者: ACaseOfIdentity:Hozma
犯人: 
犯行動機: 

In [ ]:
victim = 'ACaseOfIdentity:Hosmer'
df_ranking_ACaseOfIdentity, df_attension_ACaseOfIdentity = make_ranking(
    'ACaseOfIdentity:510', 'ACaseOfIdentity:578', HIDE, victim, MASK_E, MASK_E, MASK_E, MASK_E)

display(df_ranking_ACaseOfIdentity.iloc[:20,:])
# display(df_attension_SpeckledBand)
# ヒートマップの作成
# sns.heatmap(df_atten.iloc[:,3:].iloc[:32,:32])
len_ = len(df_attension_ACaseOfIdentity)
for i in range(len_-20, len_):
    display(i, df_attension_ACaseOfIdentity.iloc[i,:3].tolist())
    display(df_attension_ACaseOfIdentity.sort_values(f'atten_from{i}', ascending=False).iloc[:20,[0,1,2,3+i]])
    print("----------")
